In [ ]:
%load_ext autoreload
%autoreload 2

In [14]:
#needed to import qke procedures
import sys
from pathlib import Path

main_dir = str(Path().resolve().parent)
sys.path.insert(1, main_dir)

print(f'Main dir: {main_dir}')

Main dir: /home/jupyter/git/QK


In [32]:
import datetime
import sys
import os
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
from qiskit_algorithms.utils import algorithm_globals
from pqk.PQK_SVC_PE import PQK_SVC_PE
from pqk.PQK_SVC import PQK_SVC
from pqk.Circuits import Circuits
from pqk.QMeasures import QMeasures
from pqk.CKernels import CKernels

In [23]:
#quantum stuff: observable, circuit to encoding and number type of function used to measure that uses shots
my_obs = ['XIIIII', 'IXIIII','IIXIII', 'IIIXII','IIIIXI','IIIIIX',
          'YIIIII', 'IYIIII','IIYIII', 'IIIYII','IIIIYI','IIIIIY',
          'ZIIIII', 'IZIIII','IIZIII', 'IIIZII','IIIIZI','IIIIIZ']
q_c = Circuits.xyz_encoded(full_ent=False, n_wire=6)

#load dataset with panda
#data are scaled outside the notebook
f_rate = 0.01 #rate of data sampling fot testing pourpose
data_file_csv = '../data/env.sel3.sk_sc.csv'
env = pd.read_csv(data_file_csv).sample(frac=f_rate, random_state=123)  

#DEFINE design matrix
Y = env['occupancy']
X = env[['illuminance', 'blinds','lamps','rh', 'co2', 'temp']]


#split design matrix (25% of the design matrix used for test)
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=123)
#WARNING: convert data to numpy. Quantum stuff (Qiskit) do not like PANDAS
X_train_np = X_train.to_numpy()
y_train_np = y_train.to_numpy()

X_test_np = X_test.to_numpy()
y_test_np = y_test.to_numpy()

#check the shape of test and training dataset
print(f'File used for this run: {data_file_csv}')
print(f'Fraction rate used for this run: {f_rate * 100}%')
print(f'Shape of dataset: {env.shape}')
print(f'Training shape dataset {X_train_np.shape}')
print(f'Label for traing {y_train_np.shape}')
print(f'Test shape dataset {X_test_np.shape}')
print(f'Label for test {y_test_np.shape}')

File used for this run: ../data/env.sel3.sk_sc.csv
Fraction rate used for this run: 1.0%
Shape of dataset: (29, 7)
Training shape dataset (21, 6)
Label for traing (21,)
Test shape dataset (8, 6)
Label for test (8,)


In [36]:
#set the seed
np.random.seed(123)
algorithm_globals.random_seed = 123

#deinf the number of the shots
n_shots_list = range(10,50,10)

#get time
t_start = time.time()

list_score = []

for n_shot_ in n_shots_list:
    # Best paramenter: {'C': 32.0, 'gamma': 0.01}
    #pqk = PQK_SVC(C=32, gamma=0.01, circuit=q_c, obs=my_obs, c_kernel=CKernels.rbf, nshots=n_shot_)
    pqk = PQK_SVC(C=32, gamma=0.01, circuit=q_c, obs=my_obs, c_kernel=CKernels.rbf)
    
    svm_quantum = pqk.fit(X_train_np, y_train_np)
    #result...
    predictions = svm_quantum.predict(X_test_np)
    score = accuracy_score(predictions, y_test)

    list_score.append(score)

    
#get time training
t_training = time.time()


#final time (trainign + predict)
t_final = time.time()


print(f'Final time {t_final - t_start} seconds')

#save info.
np.savetxt(os.path.join(main_dir, "nhsots.txt"), np.array(n_shots_list))
np.savetxt(os.path.join(main_dir, "scores.txt"), np.array(list_score))


Final time 4.210016965866089 seconds
